# Lesson Introduction

Welcome! Today, we’ll see how to build a simple **AI agent** that answers questions using external knowledge. Even advanced AI models sometimes need to look up information — just as you might check your notes or Google before answering a tough question.

**Retrieval-Augmented Generation (RAG)** helps AI agents find and use relevant information from a knowledge base for more accurate answers. By the end, you’ll know how to create a basic agent that uses RAG to answer user queries more effectively.

## Loading and Preparing Knowledge Base

First, the agent needs access to knowledge. Here, our knowledge base is a `JSON` file with study tips and resources.

Here’s how to load it in `Python`:

```python
import os
import json

def load_data(file_name):
    """Load knowledge base from JSON file."""
    current_dir = os.path.dirname(__file__)
    dataset_file = os.path.join(current_dir, "data", file_name)
    with open(dataset_file, 'r') as file:
        return json.load(file)

data = load_data("data.json")
print(data)  # [{'id': 1, 'content': "Read Chapter 7 of 'Clean Code' — focus on writing small, single-purpose functions."}, {'id': 2, 'content': "Experiment with React's useContext by creating a theme toggler component."}, {'id': 3, 'content': 'Review different types of SQL joins — especially LEFT and FULL OUTER joins.'}, {'id': 4, 'content': 'Watch lecture on consensus algorithms — try to summarize Paxos in your own words.'}, {'id': 5, 'content': 'Figure out the difference between React Query and Redux for async data handling.'}, {'id': 6, 'content': "Draft blog post: '3 Things I Learned About Writing Better Functions.'"}, {'id': 7, 'content': 'Ask Aram about good beginner-friendly open-source projects to contribute to.'}]
```

This code locates and opens `data.json`, loading its content as a list of dictionaries. Each dictionary is a document, for example:

```python
{
    "id": 2,
    "content": "Experiment with React's useContext by creating a theme toggler component."
}
```

This simple structure works well for small knowledge sets. Larger, real-world knowledge bases follow the same principle: load and access information for the agent.

## Simple RAG Retrieval Logic

With the knowledge base loaded, the next step is retrieval — finding the most relevant document for a user’s question. We use a basic method: word overlap. We count how many words from the user’s question appear in each document. The document with the most overlap is chosen.

Here’s the code:

```python
def rag_retrieval(query, knowledge_base):
    query_words = set(query.lower().split())
    best_doc = None
    best_overlap = 0
    for doc in knowledge_base:
        doc_words = set(doc["content"].lower().split())
        overlap = len(query_words & doc_words)
        if overlap > best_overlap:
            best_overlap = overlap
            best_doc = doc
    return best_doc if best_overlap > 0 else None

# Example usage:
query = "How should I start tackling React useContext"
best_doc = rag_retrieval(query, data)
print(best_doc)  # {'id': 2, 'content': "Experiment with React's useContext by creating a theme toggler component."}
```

The query and each document are split into lowercase words, and the code counts overlapping words. The document with the highest overlap is returned. This method is simple and fast. More advanced systems use semantic search, but word overlap is a great way to start understanding retrieval.

## Building the Agent Prompt

After finding the most relevant document, we give this context to the agent. The agent uses both the user’s question and the retrieved document to answer. We build a prompt that combines both:

```python
def build_prompt(user_prompt, rag_document=None):
    if rag_document:
        return f"Based on the following document: {rag_document['content']}, {user_prompt}"
    return user_prompt

# Example usage:
user_prompt = "How should I start tackling React useContext"
prompt_with_context = build_prompt(user_prompt, best_doc)
print(prompt_with_context)  # Based on the following document: Experiment with React's useContext by creating a theme toggler component., How should I start tackling React useContext
```

If a document is found, the prompt includes its content. If not, the agent just gets the original question. Providing context helps the agent focus on useful information, leading to better answers. This is the core of RAG: retrieval plus generation.

## Running the Agent

Now, let’s see how the agent uses the prompt to generate a response. The agent is defined with instructions and run using a helper:

```python
from agents import Agent, Runner

AGENT = Agent(
    name="Learning Assistant",
    instructions=(
        "You are a personal learning assistant. "
        "Whenever asked a question about learning plans, use the context provided to answer questions."
    )
)

def ask_agent(prompt):
    try:
        result = Runner.run_sync(AGENT, prompt)
        return result.final_output
    except Exception as e:
        print(f"Agent error: {e}")
        raise

# Example usage:
response = ask_agent(prompt_with_context)
print(response)  # (Output will depend on the agent's implementation, e.g., "To start with React's useContext, try building a simple theme toggler as described in the document.")
```

The agent gets the prompt (with or without context), generates a response using its instructions and the context, and prints the response. This workflow — retrieval, prompt building, and response generation — is the foundation of a simple RAG agent.

## Lesson Summary and Practice Introduction

You’ve learned how to build a simple agent using **Retrieval-Augmented Generation (RAG)** for better answers. We covered loading a knowledge base from `JSON`, retrieving the most relevant document using word overlap, building a prompt with the user’s question and context, and running the agent to generate a response. This approach makes AI agents more helpful, especially for specific topics or documents.

Now it’s your turn! Next, you’ll practice building and running your own RAG-powered agent. You’ll load data, retrieve relevant information, and generate answers using an agent. This hands-on work will reinforce your understanding and prepare you for more advanced RAG techniques.

Great job on setting up the basic RAG retrieval! Now, let's enhance the retrieval function.

Currently, it returns only the document with the highest word overlap. Modify it to return a list of all documents that share at least one word with the query, sorted by the number of overlapping words in descending order.

This change will allow the agent to consider multiple relevant documents, thereby improving its response quality.

In [1]:
# exercise 1
import os
import json
from rag_agent import ask_agent
from agents import Runner

def load_data(file_name):
    """Load sample knowledge base content from JSON file."""
    current_dir = os.path.dirname(__file__)
    dataset_file = os.path.join(current_dir, "data", file_name)
    with open(dataset_file, 'r') as file:
        return json.load(file)
        
def rag_retrieval(query, knowledge_base):
    query_words = set(query.lower().split())
    best_doc_id = None
    best_overlap = 0
    any_match = []
    for doc in knowledge_base:
        doc_words = set(doc["content"].lower().split())
        overlap = len(query_words.intersection(doc_words))
        if overlap > 0: #revised from best_overlap:
            best_overlap = overlap
            #best_doc = doc
            any_match.append({"document":doc,"overlap_count":overlap})
    any_match = sorted(any_match, key=lambda x: x["overlap_count"], reverse=True)
    return_docs = [x["document"] for x in any_match]
    return return_docs if best_overlap > 0 else None

def build_prompt(user_prompt, rag_document_list=None):
    if rag_document_list:
        prompt_part1 = f"Based on the following documents: "
        prompt_part2 = ""
        for doc in rag_document_list:
           prompt_part2 += doc['content'] 
        return prompt_part1 + prompt_part2 + f"{user_prompt}"#f"Based on the following document: {rag_document['content']}, {user_prompt}"
    return user_prompt


def main():
    data = load_data("data.json")

    query = "How should I start tackling React useContext"

    rag_document = rag_retrieval(query, data)

    query = build_prompt(query, rag_document)

    response = ask_agent(query)

    print(response)

if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'rag_agent'

# Exercise 2

Nice work on returning multiple matching notes in the last step. Now, tighten retrieval and structure the prompt.

Add a top_k parameter to limit results to the most relevant notes by word overlap.

Then, list those notes as a bulleted Context in the prompt. If nothing matches, send the original question as is.



In [ ]:
import os
import json
from rag_agent import ask_agent
from agents import Runner

def load_data(file_name):
    """Load sample knowledge base content from JSON file."""
    current_dir = os.path.dirname(__file__)
    dataset_file = os.path.join(current_dir, "data", file_name)
    with open(dataset_file, 'r') as file:
        return json.load(file)

def rag_retrieval(query, knowledge_base, top_k=10):
    """
    Return up to top_k most relevant documents by word overlap.
    """
    # TODO: Convert the query to lowercase words and store in a set
    query_words = set(query.lower().split())

    # Collect (doc, overlap_score) for docs with at least one overlapping word
    scored = []

    # TODO: Iterate through the knowledge base and compute overlap
    for doc in knowledge_base:
        # TODO: Lowercase and split document content into words
        doc_words = set(doc["content"].lower().split())

        # TODO: Compute overlap score between query_words and doc_words
        overlap = len(query_words.intersection(doc_words))

        # TODO: Keep only documents with positive overlap
        if overlap > 0:
            scored.append((doc, overlap))

    # TODO: Sort by overlap descending
    scored.sort(key=lambda x: x[1], reverse=True)

    # TODO: Return only the top_k documents (no scores)
    return [d for d, _ in scored[:top_k]]

def build_prompt(user_prompt, rag_documents=None):
    """
    Build a prompt that lists retrieved notes under a Context section.
    If no documents are provided, return the original question.
    """
    if rag_documents:
        # TODO: Create a bullet list string from rag_documents' content lines
        context_lines = "\n".join(f"- {doc['content']}" for doc in rag_documents)
        # TODO: Compose the final prompt with question, context, and answer cue
        return f"Question: {user_prompt}\nContext:\n{context_lines}\nAnswer:"
    return user_prompt

def main():
    data = load_data("data.json")

    query = "How should I start tackling React useContext"
    top_k = 5

    rag_documents = rag_retrieval(query, data, top_k=top_k)

    prompt = build_prompt(query, rag_documents)

    response = ask_agent(prompt)

    print(response)

if __name__ == '__main__':
    main()

In [ ]:
# exercise 3
# TODO: Import needed modules:
# - os and json for reading the JSON file
# - re for simple punctuation removal
# - ask_agent from rag_agent to run the agent with a prompt



# TODO: Implement load_data(file_name):
# - Build the path to data/<file_name> using os.path
# - Open and load JSON content
# - Return the loaded list of note dicts

# TODO: Implement tokenize(text):
# - Lowercase the text
# - Remove punctuation using a regex (replace non-word, non-space with a space)
# - Split into words and return a set of unique tokens

# TODO: Implement rag_retrieval(query, knowledge_base):
# - Tokenize the query
# - For each document in the knowledge base:
#   - Tokenize doc["content"]
#   - Compute overlap size between query tokens and doc tokens
# - Track the best document by highest overlap score
# - Tie-break: if scores are equal and positive, choose the doc with smaller id
# - Return the best document, or None if there is no overlap

# TODO: Implement build_prompt(user_prompt, rag_document=None):
# - If rag_document is provided, return:
#   "Question: <user_prompt>\nContext: <rag_document['content']>\nAnswer:"
# - Otherwise, return the original user_prompt

# TODO: Implement main():
# - Load data from "data.json"
# - Define a user query like: "What should I focus on for SQL joins?"
# - Run rag_retrieval to get the best document
# - Build the prompt with build_prompt
# - Call ask_agent(prompt) and print the response

# TODO: Add the standard Python entry point guard to call main():
# if __name__ == '__main__':
#     main()


In [ ]:
# solution exercise 3
# TODO: Import needed modules:
# - os and json for reading the JSON file
# - re for simple punctuation removal
# - ask_agent from rag_agent to run the agent with a prompt
import os
import json
import re
from rag_agent import ask_agent
from agents import Runner


# TODO: Implement load_data(file_name):
# - Build the path to data/<file_name> using os.path
# - Open and load JSON content
# - Return the loaded list of note dicts
def load_data(file_name):
    """Load sample knowledge base content from JSON file."""
    current_dir = os.path.dirname(__file__)
    dataset_file = os.path.join(current_dir, "data", file_name)
    with open(dataset_file, 'r') as file:
        return json.load(file)


# TODO: Implement tokenize(text):
# - Lowercase the text
# - Remove punctuation using a regex (replace non-word, non-space with a space)
# - Split into words and return a set of unique tokens
def tokenize(text):
    text = re.sub(r'[^\w\s]+', ' ', text)
    text = text.lower().split()
    return set(text)

# TODO: Implement rag_retrieval(query, knowledge_base):
# - Tokenize the query
# - For each document in the knowledge base:
#   - Tokenize doc["content"]
#   - Compute overlap size between query tokens and doc tokens
# - Track the best document by highest overlap score
# - Tie-break: if scores are equal and positive, choose the doc with smaller id
# - Return the best document, or None if there is no overlap
def rag_retrieval(query, knowledge_base):
    query_words = tokenize(query)
    best_doc = None
    best_overlap = 0
    best_id = 0
    for doc in knowledge_base:
        doc_words = tokenize(doc["content"])
        overlap = len(query_words & doc_words)
        ids = doc["id"]
        if overlap > best_overlap:
            best_overlap = overlap
            best_doc = doc
            best_id = ids
        elif overlap == best_overlap:
            if ids < best_id:
                best_doc = doc
                best_id = ids
            
    return best_doc if best_overlap > 0 else None




# TODO: Implement build_prompt(user_prompt, rag_document=None):
# - If rag_document is provided, return:
#   "Question: <user_prompt>\nContext: <rag_document['content']>\nAnswer:"
# - Otherwise, return the original user_prompt

def build_prompt(user_prompt, rag_document=None):
    """
    Build a prompt that lists retrieved notes under a Context section.
    If no documents are provided, return the original question.
    """
    if rag_document:
        return f"Question: {user_prompt}\nContext: {rag_document['content']}\nAnswer:"
    return user_prompt
    
    
# TODO: Implement main():
# - Load data from "data.json"
# - Define a user query like: "What should I focus on for SQL joins?"
# - Run rag_retrieval to get the best document
# - Build the prompt with build_prompt
# - Call ask_agent(prompt) and print the response
def main():
    data = load_data("data.json")

    query = "How should I start tackling React useContext"
    top_k = 5

    rag_documents = rag_retrieval(query, data)

    prompt = build_prompt(query, rag_documents)

    response = ask_agent(prompt)

    print(response)


# TODO: Add the standard Python entry point guard to call main():
# if __name__ == '__main__':
#     main()
if __name__ == "__main__":
    main()